In [17]:
# ! pip install pyspark

In [18]:
import pyspark

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, year

# Création de la session Spark

In [20]:
spark = SparkSession.builder \
    .appName("Analyse des Ventes") \
    .master("local[*]") \
    .getOrCreate()

#### Création d'un exemple de données pour le test

In [21]:
donnees_test = """
2024-01-01 Paris Ordinateur 1200
2024-01-02 Lyon Telephone 800
2024-01-03 Paris Tablette 600
2024-01-04 Marseille Ordinateur 1100
2024-02-01 Lyon Telephone 750
2023-02-02 Paris Ordinateur 1300
"""

##### Écriture des données de test dans un fichier


In [22]:
with open("ventes.txt", "w") as f:
    f.write(donnees_test.strip())

### Exercice 1: Total des ventes par ville
#### Lecture du fichier

In [23]:
df = spark.read.csv("ventes.txt", sep=" ", schema="date STRING, ville STRING, produit STRING, prix DOUBLE")


#### Calcul du total des ventes par ville

In [24]:
total_ventes_ville = df.groupBy("ville") \
    .agg(sum("prix").alias("total_ventes")) \
    .orderBy("ville")

print("Résultat Exercice 1 - Total des ventes par ville:")
total_ventes_ville.show()

Résultat Exercice 1 - Total des ventes par ville:
+---------+------------+
|    ville|total_ventes|
+---------+------------+
|     Lyon|      1550.0|
|Marseille|      1100.0|
|    Paris|      3100.0|
+---------+------------+



### Exercice 2: Total des ventes par ville pour une année donnée


In [25]:
# Ajout d'une colonne année
df_with_year = df.withColumn("annee", year(df.date))

# Fonction pour calculer le total des ventes par ville pour une année spécifique
def calculer_ventes_annee(annee):
    resultat = df_with_year.filter(df_with_year.annee == annee) \
        .groupBy("ville") \
        .agg(sum("prix").alias(f"total_ventes_{annee}")) \
        .orderBy("ville")
    return resultat

# Exemple pour l'année 2024
print("\nRésultat Exercice 2 - Total des ventes par ville pour 2024:")
ventes_2024 = calculer_ventes_annee(2024)
ventes_2024.show()


Résultat Exercice 2 - Total des ventes par ville pour 2024:
+---------+-----------------+
|    ville|total_ventes_2024|
+---------+-----------------+
|     Lyon|           1550.0|
|Marseille|           1100.0|
|    Paris|           1800.0|
+---------+-----------------+



# *** Arrêt de la session Spark ***

In [26]:
spark.stop()